In [2]:
from batching import get_selected_links, get_level_tags, get_level_one_tags, batch_cycle, get_batch_responses, check_batch_status, batch_get_relevant_content
import pandas as pd
from nslookup import Nslookup
import re
from googlesearch import search
import json


In [2]:
# df = pd.read_csv('./dados/buyers/companies.csv')[['Nome','ticker', 'valor']]

In [3]:

# def corrigir_link(link: str) -> str:
    
#     if type(link) != str:
#         return ''
    
#     link = link.strip().lower().replace('http://', '').replace('https://', '').replace('/', '')
#     link = re.sub(r'-com\.br', '.com.br', link)
#     link = re.sub(r'\.combr([^a-zA-Z0-9/]|$)', r'.com.br\1', link)
#     link = link.replace('.br.com', '.com.br')
#     link = re.sub(r'\.com\.br[^/\.\s]{1,3}', '.com.br', link)
#     link = re.sub(r'\.com\.\.br', '.com.br', link)
#     link = re.sub(r'(?<!\.)com\.br', r'.com.br', link)
#     link = re.sub(r'([a-z0-9])com\.br', r'\1.com.br', link)
#     link = re.sub(r'(\.com\.br)[^\s/]*', r'\1', link)
#     link = re.sub(r'(\.com)(?!\.br|/)[^\s/]*', r'\1', link)
#     link = re.sub(r'\.{2,}', '.', link)

#     return link

# def buscar_primeiro_resultado(query: str) -> str | None:
#     try:
#         resultados = list(search(query, num_results=3))
#         return resultados[0] if 'http' in resultados[0] else resultados[1] if 'http' in resultados[1] else resultados[2] if 'http' in resultados[2] else None
#     except Exception as e:
#         print(f"Erro ao buscar: {e}")
#         return None


In [4]:
# df['valor'] = df['valor'].apply(corrigir_link)

In [5]:
# dns_query = Nslookup(verbose=False, tcp=False)
# links = {
#     'nome': [],
#     'link': []
# }
# for nome,link,ticker in zip(df['Nome'], df['valor'], df['ticker']):
#     try:
#         link = f'www.{link.strip().lower().replace('http://', '').replace('https://', '').replace('/', '')}'
#         res = dns_query.dns_lookup(link).answer
#         if len(res) < 1:
#             link = link.replace('www.', '')
#             res = dns_query.dns_lookup(link).answer
#             if len(res) < 1:
#                 link = buscar_primeiro_resultado(f'{nome} {ticker}')
#                 link = corrigir_link(link)
#                 res = dns_query.dns_lookup(link).answer
#                 if len(res) < 1:       
#                     print(f"Invalid DNS for {nome}: {link} -> {link}")

#         links['nome'].append(nome)
#         links['link'].append(link)
#     except Exception as e:
#         print(f"Error looking up {nome}: {e}")


In [6]:
# links2 = {i:j for i,j in zip(links['nome'], links['link'])}

In [7]:
# df['valor'] = df['Nome'].map(links2)
# df['valor'] = df['valor'].apply(lambda x: f'www.{x}' if not x.startswith('www.') else x)
# df['valor'] = df['valor'].apply(lambda x: f'https://{x}' if not x.startswith('http') else x)

In [8]:
# df.to_csv('./dados/buyers/companies.csv', index=False)

In [3]:
df = pd.read_csv('./dados/buyers/companies.csv')

In [10]:
# batch_file, df_links = get_selected_links(df['valor'].unique().tolist(), validate=False, max_workers=5)

In [11]:
jsonObj = pd.read_json(path_or_buf='./batch_file_1754071963_556955.jsonl', lines=True)
jsonObj['input_links'] = jsonObj['body'].apply(lambda x: x['messages'][1]['content'])
df['input_links'] = df['valor'].apply(lambda x: jsonObj[jsonObj['custom_id'] == x]['input_links'].values[0] if x in jsonObj['custom_id'].values else None)
df['input_links'] = df['input_links'].apply(lambda x: x.split(';') if isinstance(x, str) else '')

In [12]:
# results = batch_cycle(batch_file)
results = get_batch_responses(check_batch_status('batch_688d03fd20a88190a0393b2621b6be6b'))

[27/08/25 14:44:39] Batch Id: batch_688d03fd20a88190a0393b2621b6be6b,  Status: completed, Elapsed time 487865:44:39	


In [13]:
t = pd.DataFrame(results)
t['response'] = t['response'].apply(lambda x: json.loads(x['body']['choices'][0]['message']['content'])['links'])

In [14]:
df = df.merge(
    t[['custom_id','response']],
    how='left',
    left_on='valor',
    right_on='custom_id'
)

In [15]:
df['input_links'] = df['input_links'].apply(lambda x: x if isinstance(x, list) else [])
df['response'] = df['response'].apply(lambda x: x if isinstance(x, list) else [])

df['good_links'] = df.apply(lambda x: list(set(x['response']).intersection(set(x['input_links']))), axis=1)

In [16]:
# contents = batch_get_relevant_content(df[['valor','good_links']].to_dict(orient='records'))
# with open('./contents.txt', 'w') as f:
#     json.dump(contents, f, indent=4)

In [24]:
with open('./contents.json', 'r') as f:
    contents = json.load(f)

In [56]:
from datetime import datetime

from tqdm import tqdm
from batching import RESPONSE_FORMATS, SUMMARY_PROMPT, compose_batch_file, create_batch_item, get_reference_object
from scrapping import clean_pages


def batch_sumirize_company(pages: list[dict], model: str = 'gpt-4o-mini-2024-07-18') -> list[dict]:
    batch_file = f'batch_file_{str(datetime.now().timestamp()).replace(".","_")}.jsonl'
    json_reference = get_reference_object()

    for i, (url, page) in enumerate(pages.items()):
        if not page:
            continue
        page = clean_pages(page)
        context = ';'.join(f'{link} -> {content}' for link, content in page.items())
        item = create_batch_item(url, SUMMARY_PROMPT, context, RESPONSE_FORMATS['SUMMARY'], json_reference, model='gpt-4.1-mini-2025-04-14')
        compose_batch_file(item, batch_file)

    print(f"Batch file created: {batch_file}")
    print("Starting batch processing...")
    return batch_cycle(batch_file, delete=False)


# response = batch_sumirize_company(contents)

In [63]:
json.loads(response[0]['response']['body']['choices'][0]['message']['content'])['summary']
# response[0]['custom_id']
t = pd.DataFrame(response)
t['resumo'] = t['response'].apply(lambda x: json.loads(x['body']['choices'][0]['message']['content'])['summary'])

In [65]:
# Merge the response list (as DataFrame t) with df on 'custom_id'
df_matched = df.merge(
    t[['custom_id', 'resumo']],
    how='left',
    on='custom_id',
    suffixes=('', '_from_response')
)

In [75]:
custom_ids_in_t = set(t['custom_id'])

urls_in_contents = set(contents.keys())

missing_urls = urls_in_contents - custom_ids_in_t

missing_entries = {url: contents[url] for url in missing_urls}

In [ ]:
response = batch_sumirize_company(missing_entries)

Batch file created: batch_file_1756323423_673113.jsonl
Starting batch processing...
[27/08/25 16:41:28] Batch Id: batch_68af5e66d7648190b08633a675c8b411,  Status: completed, Elapsed time 00:04:17			


In [77]:
t2 = pd.DataFrame(response)

t2['resumo'] = t2['response'].apply(lambda x: json.loads(x['body']['choices'][0]['message']['content'])['summary'])

In [79]:
df_matched2 = df_matched.merge(
    t2[['custom_id', 'resumo']],
    how='left',
    on='custom_id',
    suffixes=('', '_from_response')
)


In [85]:
df_matched2.to_csv('df_matched2.csv', index=False)

In [80]:
custom_ids_in_t = set(t2['custom_id'])

urls_in_contents = set(contents.keys())

missing_urls = urls_in_contents - custom_ids_in_t

missing_entries = {url: contents[url] for url in missing_urls}

In [13]:
df_matched2 = pd.read_csv('./df_matched2.csv')

In [14]:
# response = batch_sumirize_company(missing_entries)
response = get_batch_responses(check_batch_status('batch_68af686ed9d88190ad4d079f2ec895c5'))

[28/08/25 18:19:50] Batch Id: batch_68af686ed9d88190ad4d079f2ec895c5,  Status: completed, Elapsed time 487893:19:50	


In [15]:
t3 = pd.DataFrame(response)

t3['resumo'] = t3['response'].apply(lambda x: json.loads(x['body']['choices'][0]['message']['content'])['summary'])
df_matched3 = df_matched2.merge(
    t3[['custom_id', 'resumo']],
    how='left',
    on='custom_id',
    suffixes=('', '_from_response_3')
)


In [63]:
df_matched3['resumo_final'] = df_matched3.apply(lambda x: x['resumo_from_response'] if pd.isna(x['resumo']) else x['resumo'], axis=1)

In [64]:
df_matched3['resumo_final'] = df_matched3.apply(lambda x: x['resumo_from_response_3'] if pd.isna(x['resumo_final']) else x['resumo_final'], axis=1)

In [57]:
missing_entries = {url: contents[url] for url in df_matched3[(df_matched3['resumo_final'].isna())&(df_matched3['good_links'].str.len() > 2)]['valor'].unique()}

response = batch_sumirize_company(missing_entries)

Batch file created: batch_file_1756417017_887284.jsonl
Starting batch processing...
[28/08/25 18:40:51] Batch Id: batch_68b0cbffc0088190986c6e7fa872ce63,  Status: completed, Elapsed time 00:03:47			


In [65]:
df_matched3

,Nome,ticker,valor,input_links,custom_id,response,good_links,resumo,resumo_from_response,resumo_from_response_3,resumo_final
0,BRF,brfs3,https://www.brf-global.com,['https://www.brf-global.com/transparenciabrf/...,https://www.brf-global.com,['https://www.brf-global.com/sobre/qualidade/'...,['https://www.brf-global.com/sobre/a-brf/nossa...,BRF is one of the largest food companies globa...,NaN,NaN,BRF is one of the largest food companies globa...
1,B3,b3sa3,https://www.b3.com.br,['https://www.b3.com.br'],https://www.b3.com.br,['https://www.b3.com.br'],['https://www.b3.com.br'],B3 is a Brazilian company that operates as one...,NaN,NaN,B3 is a Brazilian company that operates as one...
2,Ambev,abev3,https://www.ambev.com.br,['https://www.ambev.com.br'],https://www.ambev.com.br,"['https://www.ambev.com.br/sobre-nos', 'https:...",[],NaN,NaN,NaN,NaN
3,Cogna,cogn3,https://www.cogna.com.br,"['https://www.cogna.com.br', 'https://www.cogn...",https://www.cogna.com.br,"['https://www.cogna.com.br', 'https://www.cogn...","['https://www.cogna.com.br/imprensa/', 'https:...",Cogna Educação is a leading Brazilian educatio...,NaN,NaN,Cogna Educação is a leading Brazilian educatio...
4,Banco do Brasil,bbas3,https://www.bb.com.br,[],NaN,[],[],NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
393,Tronox Pigmentos,crpg3,https://www.cristal-al.com.br,['https://cristal-al.com.br/produtos-de-beleza...,https://www.cristal-al.com.br,"['https://cristal-al.com.br/', 'https://www.cr...","['https://cristal-al.com.br/', 'https://crista...",NaN,NaN,Cristal Sustentabilidade is a company focused ...,Cristal Sustentabilidade is a company focused ...
394,Iguatemi,igti4,https://www.jereissati.com.br,"['https://ri.iguatemi.com.br/fale-com-ri', 'ht...",https://www.jereissati.com.br,"['https://ri.iguatemi.com.br/fale-com-ri', 'ht...",['https://ri.iguatemi.com.br/composicao-aciona...,Iguatemi S.A. is a Brazilian company specializ...,NaN,NaN,Iguatemi S.A. is a Brazilian company specializ...
395,Jereissati Participações,igti4,https://www.jereissati.com.br,"['https://ri.iguatemi.com.br/fale-com-ri', 'ht...",https://www.jereissati.com.br,"['https://ri.iguatemi.com.br/fale-com-ri', 'ht...",['https://ri.iguatemi.com.br/composicao-aciona...,Iguatemi S.A. is a Brazilian company specializ...,NaN,NaN,Iguatemi S.A. is a Brazilian company specializ...
396,Iguatemi,igti4,https://www.jereissati.com.br,"['https://ri.iguatemi.com.br/fale-com-ri', 'ht...",https://www.jereissati.com.br,"['https://ri.iguatemi.com.br/fale-com-ri', 'ht...",['https://ri.iguatemi.com.br/composicao-aciona...,Iguatemi S.A. is a Brazilian company specializ...,NaN,NaN,Iguatemi S.A. is a Brazilian company specializ...


In [66]:
t4 = pd.DataFrame(response)

t4['resumo'] = t4['response'].apply(lambda x: json.loads(x['body']['choices'][0]['message']['content'])['summary'])
df_matched4 = df_matched3.merge(
    t4[['custom_id', 'resumo']],
    how='left',
    on='custom_id',
    suffixes=('', '_from_response_4')
)


In [68]:
df_matched4['resumo_final'] = df_matched4.apply(lambda x: x['resumo_from_response_4'] if pd.isna(x['resumo_final']) else x['resumo_final'], axis=1)

In [72]:
df_matched4[~df_matched4['resumo_final'].isna()][['Nome', 'ticker', 'valor', 'resumo_final']].to_csv('./dados/buyers/companies_with_summaries.csv', index=False)